# Heuristic Analysis for Minimax AI in Othello

In this notebook, we explore the most effective **heuristics** for guiding the Minimax algorithm. 

Several AI agents, each employing different combinations of heuristics, engage in head-to-head matches. Through these matches, we identify the strongest heuristics that lead to superior AI performance and strategic gameplay.

---

## Theoretical Analysis

The theoretical analysis in this notebook focuses on match evaluation. Specifically, we explore which AI agents (in terms of their heursitics, heuristic weights, and depth settings) achieve the most wins. Based on these results, we infer the strategies that lead to superior Othello AI agents.   

Hypotheses:

- Corner control is the most important heuristic [1], since gaining corner squares are fundamental to Othello strategy [2]. 
- Combining all three heuristics (disc difference, mobility, and corner control) should yield the strongest AI [1].

Methodology:

- Several AI agents are assessed, each using different combinations of heuristics, via. **head-to-head matches**.
- To ensure fairness, every agent competes against every other agent *twice*, once as Black and once as White.
- To ensure fairness, every Minimax agent has depth set to three, i.e. $d=3$. Additionally, this is done for practicality, i.e. computational efficiency.
- Results naturally generate a **league structure**, akin to the common format used in football leagues worldwide, following a "double round-robin" basis.
- To maintain simplicity, for most agents we employ uniform weights and reserve weight optimization for future analyses, i.e. fine-tuning AI performance.

**Note:** For `agent_8`, the weights used are motivated by [1]. Sannidhanam & Muthukaruppan briefly discuss the weights they used, which were inspired by *"a lot of experimentation against online computer Othello players"*. In this Othello project, we explore their weights, albeit with adjustments, as we have not included a "stability" heuristic as they did.

### References

[1] Sannidhanam, A., & Muthukaruppan, A. (2004). '[An Analysis of Heuristics in Othello](https://courses.cs.washington.edu/courses/cse573/04au/Project/mini1/RUSSIA/Final_Paper.pdf)'.

[2] Rose, B. (2004). '[Othello: A Minute to Learn... A Lifetime to Master.](https://www.ffothello.org/livres/othello-book-Brian-Rose.pdf)' Anjar Co.

---

## Load Source Code

In [1]:
import os
import sys

# Set the working directory to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
os.chdir(project_root)

from othello.src.game import Game
from othello.src.board import SquareType
from othello.src.player import Player, PlayerType
from othello.src.state_evaluation import StateEvaluator, HeuristicType
from tqdm import tqdm

## Initialise Players

In [2]:
# Agents dictionary
agents = {}
depth = 3

# Agent 1: Disc difference
custom_weights = {HeuristicType.DISC_DIFF: 1.0}
state_eval = StateEvaluator(weights=custom_weights)
agents['agent_1'] = {
    'black': Player(PlayerType.MINIMAX, SquareType.BLACK, state_eval, depth),
    'white': Player(PlayerType.MINIMAX, SquareType.WHITE, state_eval, depth)
}

# Agent 2: Mobility
custom_weights = {HeuristicType.MOBILITY: 1.0}
state_eval = StateEvaluator(weights=custom_weights)
agents['agent_2'] = {
    'black': Player(PlayerType.MINIMAX, SquareType.BLACK, state_eval, depth),
    'white': Player(PlayerType.MINIMAX, SquareType.WHITE, state_eval, depth)
}

# Agent 3: Corners
custom_weights = {HeuristicType.CORNERS: 1.0}
state_eval = StateEvaluator(weights=custom_weights)
agents['agent_3'] = {
    'black': Player(PlayerType.MINIMAX, SquareType.BLACK, state_eval, depth),
    'white': Player(PlayerType.MINIMAX, SquareType.WHITE, state_eval, depth)
}

# Agent 4: Disc difference & Corners
custom_weights = {HeuristicType.DISC_DIFF: 0.5, HeuristicType.CORNERS: 0.5}
state_eval = StateEvaluator(weights=custom_weights)
agents['agent_4'] = {
    'black': Player(PlayerType.MINIMAX, SquareType.BLACK, state_eval, depth),
    'white': Player(PlayerType.MINIMAX, SquareType.WHITE, state_eval, depth)
}

# Agent 5: Disc difference & Mobility
custom_weights = {HeuristicType.DISC_DIFF: 0.5, HeuristicType.MOBILITY: 0.5}
state_eval = StateEvaluator(weights=custom_weights)
agents['agent_5'] = {
    'black': Player(PlayerType.MINIMAX, SquareType.BLACK, state_eval, depth),
    'white': Player(PlayerType.MINIMAX, SquareType.WHITE, state_eval, depth)
}

# Agent 6: Mobility & Corners
custom_weights = {HeuristicType.MOBILITY: 0.5, HeuristicType.CORNERS: 0.5}
state_eval = StateEvaluator(weights=custom_weights)
agents['agent_6'] = {
    'black': Player(PlayerType.MINIMAX, SquareType.BLACK, state_eval, depth),
    'white': Player(PlayerType.MINIMAX, SquareType.WHITE, state_eval, depth)
}

# Agent 7: All Heuristics
custom_weights = {
    HeuristicType.DISC_DIFF: 1/3,
    HeuristicType.MOBILITY: 1/3,
    HeuristicType.CORNERS: 1/3
}
state_eval = StateEvaluator(weights=custom_weights)
agents['agent_7'] = {
    'black': Player(PlayerType.MINIMAX, SquareType.BLACK, state_eval, depth),
    'white': Player(PlayerType.MINIMAX, SquareType.WHITE, state_eval, depth)
}

# Agent 8: All Heuristics 
# Weights motivated by 'An Analysis of Heuristics in Othello'
custom_weights = {
    HeuristicType.DISC_DIFF: 25/60,
    HeuristicType.MOBILITY: 5/60,
    HeuristicType.CORNERS: 30/60
}
state_eval = StateEvaluator(weights=custom_weights)
agents['agent_8'] = {
    'black': Player(PlayerType.MINIMAX, SquareType.BLACK, state_eval, depth),
    'white': Player(PlayerType.MINIMAX, SquareType.WHITE, state_eval, depth)
}

## Simulate Match Function

- A function to simulate an Othello match between two agents.
- Returns all relevant information, i.e winnner, score etc.

In [3]:
def simulate_match(agent_black_name, agent_white_name):
    """
    Simulates an Othello match between two agents.
    """
    
    # Get agent using dictionary
    agent_black = agents.get(agent_black_name)
    agent_white = agents.get(agent_white_name)
    
    # Instantiate game instance
    player_black = agent_black["black"]
    player_white = agent_white["white"]
    game = Game(player_black, player_white)
    
    while not game.is_finished:
        game.get_player_move() 
        game.make_move()
        game.change_turn()
        game.update_valid_moves()
        game.update_scores()
        game.check_finished()
    
    game.determine_winner()

    return {
        'game_result': game.game_result,
        'black_score': game.black_score,
        'white_score': game.white_score,
        'agent_black': agent_black_name,
        'agent_white': agent_white_name
    }

## Generate (All) Match Results Function

- A function to generate match results for each agent matchup. 
- Returns all relevant information, i.e winnner, score etc.

In [4]:
def generate_match_results():
    """
    Generates match results for each agent matchup. 
    """
    
    match_results = []
    agent_names = list(agents.keys())

    for i in tqdm(range(len(agent_names))):
        for j in tqdm(range(i + 1, len(agent_names))):
            print(f"Playing: {agent_names[i]} vs. {agent_names[j]}")
            
            result_1 = simulate_match(agent_names[i], agent_names[j])
            result_2 = simulate_match(agent_names[j], agent_names[i])
            
            match_results.append(result_1)
            match_results.append(result_2)

    return match_results

- Run the above, generating all match results.
- Save locally and interpret results.

In [5]:
# Generate match results
match_results = generate_match_results()

  0%|                                                     | 0/7 [00:00<?, ?it/s]

Playing: agent_1 vs. agent_2



 14%|██████▎                                     | 1/7 [02:30<15:02, 150.47s/it]

Playing: agent_1 vs. agent_3



 29%|████████████▊                                | 2/7 [03:23<07:45, 93.05s/it]

...


 50%|██████████████████████                      | 1/2 [04:47<04:47, 287.58s/it]

Playing: agent_6 vs. agent_8



  0%|                                                     | 0/1 [00:00<?, ?it/s]

Playing: agent_7 vs. agent_8



 88%|████████████████████████████████████▊     | 7/8 [1:32:09<11:01, 661.76s/it]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████| 8/8 [1:32:09<00:00, 691.19s/it]


In [6]:
import pandas as pd

# Save Data Frame locally, as .csv
df_results = pd.DataFrame(match_results)
df_results.to_csv("match_results.csv")
df_results.head()

,game_result,black_score,white_score,agent_black,agent_white
0,Black Wins,37,0,agent_1,agent_2
1,White Wins,26,38,agent_2,agent_1
2,White Wins,14,50,agent_1,agent_3
3,White Wins,22,42,agent_3,agent_1
4,White Wins,21,43,agent_1,agent_4


- Transform the match results DataFrame into a league table.

In [7]:
# Create a list of unique agent names
all_agents = pd.concat([df_results['agent_black'], df_results['agent_white']]).unique()

# Initialize a DataFrame for the league table
df_league = pd.DataFrame(index=all_agents)

# Count wins, draws, and losses for each agent
df_league['Wins'] = (
    (df_results['game_result'] == 'Black Wins').groupby(df_results['agent_black']).sum() +
    (df_results['game_result'] == 'White Wins').groupby(df_results['agent_white']).sum()
)

df_league['Draws'] = (
    (df_results['game_result'] == 'Draw').groupby(df_results['agent_black']).sum() +
    (df_results['game_result'] == 'Draw').groupby(df_results['agent_white']).sum()
)

df_league['Losses'] = (
    (df_results['game_result'] == 'White Wins').groupby(df_results['agent_black']).sum() +
    (df_results['game_result'] == 'Black Wins').groupby(df_results['agent_white']).sum()
)

# Calculate points based on wins and draws
df_league['Points'] = df_league['Wins'] * 3 + df_league['Draws']

# Fill NaN values with 0 for agents with no matches
df_league.fillna(0, inplace=True)

# Sort the league table by Points in descending order
df_league.sort_values(by='Points', ascending=False, inplace=True)

- Save locally and interpret results.

In [8]:
# Save Data Frame locally, as .csv
df_league.to_csv("league_table.csv")
df_league

,Wins,Draws,Losses,Points
agent_8,9,1,4,28
agent_6,8,0,6,24
agent_7,8,0,6,24
agent_4,7,0,7,21
agent_5,7,0,7,21
agent_1,6,1,7,19
agent_2,6,0,8,18
agent_3,4,0,10,12


**Main Finding:**
    
- Agent `agent_8` was the strongest-performing AI, reinforcing the results found by Sannidhanam & Muthukaruppan [1]. 

Other Findings:

- Interestingly, corner control by itself was the worst-performing AI, suggesting the importance of other heuristics.
- Generally, agents with more heuristics performed better, motivating all heuristics explored. 